In [1]:
import sys
import os
sys.path.append(os.path.dirname(os.getcwd()))

In [2]:
import numpy as np
import bartz
from pyinstrument import Profiler
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, root_mean_squared_error
from bart_playground import *
import arviz as az

INFO:arviz.preview:arviz_base not installed
INFO:arviz.preview:arviz_stats not installed
INFO:arviz.preview:arviz_plots not installed


In [3]:
generator = DataGenerator(n_samples=200, n_features=2, noise=0.1, random_seed=42)
X, y = generator.generate(scenario="piecewise_flat")
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [4]:
proposal_probs = {"multi_grow": 0.25, "multi_prune": 0.25, "multi_change": 0.4, "multi_swap": 0.1}

bart = MultiBART(ndpost=200, nskip=100, n_trees=100, proposal_probs=proposal_probs, multi_tries=10)
bart.fit(X_train, y_train)

Iterations: 100%|██████████| 300/300 [00:07<00:00, 39.29it/s]


In [5]:
import pandas as pd

# Collect move counts
selected = bart.sampler.move_selected_counts
success = bart.sampler.move_success_counts
accepted = bart.sampler.move_accepted_counts

# Combine into a DataFrame for easy viewing
df = pd.DataFrame({
    "selected": pd.Series(selected),
    "success": pd.Series(success),
    "accepted": pd.Series(accepted)
})

# Add success and acceptance rates
df["success_rate"] = df["success"] / df["selected"]
df["mh_ratio"] = df["accepted"] / df["success"]
df["accept_rate"] = df["accepted"] / df["selected"]

print(df)

              selected  success  accepted  success_rate  mh_ratio  accept_rate
multi_grow        7423     7409      3185      0.998114  0.429883     0.429072
multi_prune       7725     7602      2925      0.984078  0.384767     0.378641
multi_change     11908    11729      8369      0.984968  0.713531     0.702805
multi_swap        2944     1703       977      0.578465  0.573693     0.331861


## Default BART

In [6]:
proposal_probs = {"grow": 0.25, "prune": 0.25, "change": 0.4, "swap": 0.1}
bart = DefaultBART(ndpost=200, nskip=100, n_trees=100, proposal_probs=proposal_probs)
bart.fit(X_train, y_train)

Iterations: 100%|██████████| 300/300 [00:01<00:00, 250.47it/s]


In [7]:
import pandas as pd

# Collect move counts
selected = bart.sampler.move_selected_counts
success = bart.sampler.move_success_counts
accepted = bart.sampler.move_accepted_counts

# Combine into a DataFrame for easy viewing
df = pd.DataFrame({
    "selected": pd.Series(selected),
    "success": pd.Series(success),
    "accepted": pd.Series(accepted)
})

# Add success and acceptance rates
df["success_rate"] = df["success"] / df["selected"]
df["mh_ratio"] = df["accepted"] / df["success"]
df["accept_rate"] = df["accepted"] / df["selected"]

print(df)

        selected  success  accepted  success_rate  mh_ratio  accept_rate
grow        7411     7411      2838      1.000000  0.382944     0.382944
prune       7450     6980      2692      0.936913  0.385673     0.361342
change     12076    11280      6419      0.934084  0.569060     0.531550
swap        3063      844       538      0.275547  0.637441     0.175645
